In [1]:
import pandas as pd
import numpy as np
import requests
from pybliometrics.scopus import AuthorRetrieval
from pybliometrics.scopus import ScopusSearch
from selenium import webdriver
from IPython.display import display, Math, Latex
import re

In [4]:
# This cell and the next one retrieves the names of a faculty's academics.

try :
    driver = webdriver.Chrome(#path to chromedriver.exe)
    urlpage = 'https://isid.research.ac.ir/' 
    driver.get(urlpage)
except NameError : 
    print ("Install webdrive properly with pip")
except :
    print ("You do not have chromedriver in your path, or you should use a new version of chromedriver.\nDownload and unzip it in your desired path : https://chromedriver.chromium.org/")

In [5]:
# Set the ISID page to your desired school and faculty page.

content = [i for i in set(map(lambda x : x.get_attribute('href'),driver.find_elements_by_css_selector('a.zar'))) if 'University' not in i]
authors = list(map(lambda x : re.findall ('authorId=(\d+)', requests.get (x).text)[0], content))

In [6]:
# This cell makes a dataframe of each scholar's papers. The dataframe includes the paper's title,
# the publisher and the number of citations
auth_papers = [ScopusSearch('AU-ID  ({})'.format(i),view = 'COMPLETE') for i in authors]
auth_papers_dfs = [pd.DataFrame(i.results)[['title','publicationName','citedby_count']] for i in auth_papers]
auth_papers_dfss = auth_papers_dfs.copy ()

In [ ]:
# SCImago Journal Rank datas up to 2019
df = pd.read_table("scimagojr 2019.csv", sep=";")
journals = df[['Title','SJR','SJR Best Quartile','H index']]

In [8]:
# Adding some journals metrics to authors dataframe
auth_metrics = [x.merge(journals,left_on='publicationName' ,right_on='Title',how='left').drop (['Title'], axis=1) for x in auth_papers_dfs]
for i in auth_metrics :
    i['SJR'].replace(',','.',regex=True,inplace=True)
    i['SJR Best Quartile'].replace('Q','',regex=True,inplace=True)
    i['SJR'] = pd.to_numeric(i['SJR'], downcast='float')
    i['SJR Best Quartile'] = pd.to_numeric(i['SJR Best Quartile'], downcast='float',errors='coerce')
    i['citedby_count'] = pd.to_numeric(i['citedby_count'], downcast='float')

In [9]:
scholars = len(auth_metrics)
papers = sum([len(x) for x in auth_metrics])
print ("The dataset includes {papers} papers of {scholars} faculty members of Shiraz School of Pharmacy".format (papers = papers, scholars = scholars))

The dataset includes 1965 papers of 38 faculty members of Shiraz School of Pharmacy


In [27]:
#n = [[y,x['SJR'].mean(),x['SJR'].std(),x['H index'].mean(),x['H index'].std(),x['PI'].mean(),x['PI'].std(),x['SJR Best Quartile'].mean(),x['SJR Best Quartile'].std()] for y, x in zip(authors,auth_metrics)]
#school_metrics = pd.DataFrame (n,columns = ['Scopus ID', 'SJR mean', 'SJR SD', 'H index mean', 'H index SD','PI mean', 'PI SD','SJR Best Quartile mean', 'SJR Best Quartile SD'])